## Plant Pathology 2021 - FGVC8
#### Identify the category of foliar diseases in apple trees

Apples are one of the most important temperate fruit crops in the world. Foliar (leaf) diseases pose a major threat to the overall productivity and quality of apple orchards. The current process for disease diagnosis in apple orchards is based on manual scouting by humans, which is time-consuming and expensive.

Although computer vision-based models have shown promise for plant disease identification, there are some limitations that need to be addressed. Large variations in visual symptoms of a single disease across different apple cultivars, or new varieties that originated under cultivation, are major challenges for computer vision-based disease identification. These variations arise from differences in natural and image capturing environments, for example, leaf color and leaf morphology, the age of infected tissues, non-uniform image background, and different light illumination during imaging etc.

#### Importing libraries

In [ ]:
import numpy as np 
import pandas as pd
from pathlib import Path
import os.path
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os
import cv2

from keras.callbacks import EarlyStopping

In [ ]:
# Create a list with the filepaths for training and testing
train_img_Path = '../input/plant-pathology-2021-fgvc8/train_images'

test_img_Path = '../input/plant-pathology-2021-fgvc8/test_images'

img_Path = '../input/resized-plant2021/img_sz_256'

train = pd.read_csv(r'../input/plant-pathology-2021-fgvc8/train.csv')

sample_submission = pd.read_csv(r'../input/plant-pathology-2021-fgvc8/sample_submission.csv')

In [ ]:
train.head()

In [ ]:
print(f'Number of pictures in the training dataset: {train.shape[0]}\n')
print(f'Number of different labels: {len(train.labels.unique())}\n')
print(f'Labels: {train.labels.unique()}')

In [ ]:
train['labels'].value_counts()

In [ ]:
plt.figure(figsize=(14,7))
b = sns.countplot(x='labels', data=train, order=sorted(train['labels'].unique()))
for item in b.get_xticklabels():
    item.set_rotation(90)
plt.title('Label Distribution', weight='bold')
plt.show()

In [ ]:
plt.figure(figsize=(20,40))
i=1
for idx,s in train.head(9).iterrows():
    img_path = os.path.join(img_Path,s['image'])
    img=cv2.imread(img_path)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    fig=plt.subplot(9,3,i)
    fig.imshow(img)
    fig.set_title(s['labels'])
    i+=1

In [ ]:
CLASSES = train['labels'].unique().tolist()

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# Preprocessing the Training set
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range = 0.1,
                                   zoom_range = 0.1,
                                   horizontal_flip = True,
                                   validation_split=0.25)

train_data = train_datagen.flow_from_dataframe(train,
                                              directory=img_Path,
                                              classes=CLASSES,
                                              x_col="image",
                                              y_col="labels",
                                              target_size=(150, 150),
                                              subset='training')

val_data = train_datagen.flow_from_dataframe(train,
                                            directory=img_Path,
                                            classes=CLASSES,
                                            x_col="image",
                                            y_col="labels",
                                            target_size=(150, 150),
                                            subset='validation')

In [ ]:
dict_classes = train_data.class_indices
dict_classes

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import load_img
from keras.utils import to_categorical
from keras import Sequential
from keras.applications import InceptionResNetV2, DenseNet169, ResNet152V2
from tensorflow.keras.layers import Dense

* I am using the ResNet152V2, InceptionResNetV2, DenseNet169.

#### Defining the ResNet152V2 Convolutional Neural Net:

In [ ]:
base_Net = ResNet152V2(include_top = False, 
                         weights = '../input/keras-pretrained-models/ResNet152V2_NoTop_ImageNet.h5', 
                         input_shape = train_data.image_shape, 
                         pooling='avg',
                         classes = CLASSES)

In [ ]:
#Callbacks
EarlyStop_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
my_callback=[EarlyStop_callback]

In [ ]:
#Adding the final layers to the above base models where the actual classification is done in the dense layers
model_Net = Sequential()
model_Net.add(base_Net)
model_Net.add(Dense(12, activation=('softmax')))

model_Net.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['AUC'])
model_Net.summary()

# Training the CNN on the Train data and evaluating it on the val data
b = model_Net.fit(train_data, validation_data = val_data, epochs = 10,callbacks=my_callback, batch_size=128)

#### Defining the InceptionResNetV2 Convolutional Neural Net:

In [ ]:
base_InceptionResNetV2 = InceptionResNetV2(include_top = False, 
                         weights = '../input/keras-pretrained-models/InceptionResNetV2_NoTop_ImageNet.h5', 
                         input_shape = train_data.image_shape, 
                         pooling='avg',
                         classes = CLASSES)

In [ ]:
#Adding the final layers to the above base models where the actual classification is done in the dense layers
model_IResNet2 = Sequential()
model_IResNet2.add(base_InceptionResNetV2)
model_IResNet2.add(Dense(12, activation=('softmax')))

model_IResNet2.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model_IResNet2.summary()

# Training the CNN on the Train data and evaluating it on the val data
c = model_IResNet2.fit(train_data, validation_data = val_data, epochs = 10,callbacks=my_callback, batch_size=128)

#### Defining the DenseNet169 Convolutional Neural Net:

In [ ]:
base_DenseNet169 = DenseNet169(include_top = False, 
                         weights = '../input/keras-pretrained-models/DenseNet169_NoTop_ImageNet.h5', 
                         input_shape = train_data.image_shape, 
                         pooling='avg',
                         classes = CLASSES)

In [ ]:
#Adding the final layers to the above base models where the actual classification is done in the dense layers
model_dense = Sequential()
model_dense.add(base_DenseNet169)
model_dense.add(Dense(12, activation=('softmax')))

model_dense.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model_dense.summary()

# Training the CNN on the Train data and evaluating it on the val data
d = model_dense.fit(train_data, validation_data = val_data, epochs = 10,callbacks=my_callback, batch_size=128)

In [ ]:
test_dir = '/kaggle/input/plant-pathology-2021-fgvc8/test_images/'
test_df = pd.DataFrame()
test_df['image'] = os.listdir(test_dir)

test_data = train_datagen.flow_from_dataframe(dataframe=test_df,
                                    directory=test_dir,
                                    x_col="image",
                                    y_col=None,
                                    batch_size=32,
                                    seed=42,
                                    shuffle=False,
                                    class_mode=None,
                                    target_size=(150, 150))

#### Making predictions on test data:

In [ ]:
pred_net = model_Net.predict(test_data)
pred_iresnet2 = model_IResNet2.predict(test_data)
pred_dense = model_dense.predict(test_data)

In [ ]:
pred = ((pred_net+pred_iresnet2+pred_dense)/3).tolist()

In [ ]:
for i in range(len(pred)):
    pred[i] = np.argmax(pred[i])

    
def get_key(val):
    for key, value in dict_classes.items():
        if val == value:
            return key
        

for i in range(len(pred)):
    pred[i] = get_key(pred[i])

In [ ]:
test_df['labels'] = pred
test_df.to_csv('submission.csv',index=False)

#### If you liked the Notebook Please Upvote It !
#### Thank You